<a href="https://colab.research.google.com/github/Mohneesh09/Trend-Rafa/blob/main/best_trendline_detection_automated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from urllib.request import urlopen 
import json
from scipy.signal import argrelextrema
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.linear_model import RANSACRegressor
from statsmodels.nonparametric.kernel_regression import KernelReg
#Reducing data (pending) Window Approach. 5,3,1 yrs,6 months
#DBSCAN method on y-axis(clustering nearby pivots and rating of clusters).  #pd ranking fxn

In [2]:
def load_data(symbol):
  ticker=symbol   #QQQ
  API_KEY = "717311313634cf72e5c3b021728afe4e"
  url = "https://financialmodelingprep.com/api/v3/historical-price-full/{}?apikey={}".format(ticker, API_KEY)
  
  # store the response of URL
  response = urlopen(url)
  
  # storing the JSON response 
  data_json = json.loads(response.read()) 
  df=pd.DataFrame(data_json['historical'])
  return df

In [3]:
#https://medium.com/@chris_42047/3-ways-to-smooth-the-price-waves-with-python-9a4e170aaf15
#Will use Nadaraya–Watson kernel regression for smoothing prices.
def smoothed_curve(df_close,bandwidth):
  kernel_reg= KernelReg([df_close.values],[df_close.index],var_type='c',bw=bandwidth)
  reg_result = kernel_reg.fit([df_close.index])
  smoothed_df_close = pd.Series(data=reg_result[0], index=df_close.index)

  return smoothed_df_close

In [7]:
smoothed_df_close=smoothed_curve(df['close'],bandwidth='cv_ls')
smoothed_df_low=smoothed_curve(df['low'],bandwidth='cv_ls')
smoothed_df_high=smoothed_curve(df['high'],bandwidth='cv_ls')

In [59]:
df1= df[658:1258]
#smoothed_df_close=smoothed_df_close[658:1258]

In [60]:
df.columns

Index(['date', 'close', 'open', 'high', 'low'], dtype='object')

In [8]:
def find_extrema(df):
    from scipy.signal import argrelextrema
    #Calculating local min and max
    smoothed_local_max = argrelextrema(df['high'].values, np.greater, order=15)[0]
    smoothed_local_min = argrelextrema(df['low'].values, np.less, order=15)[0]

    smoothed_max_list = []
    smoothed_min_list = []
    max_id = 0
    min_id = 0
    for i in range(df.index.start, df.index.stop+1, 1):
        if max_id < len(smoothed_local_max):
            if i-df.index.start == smoothed_local_max[max_id]:
                smoothed_max_list.append(df['high'][i])
                max_id += 1
            else:
                smoothed_max_list.append(np.nan)
        if min_id < len(smoothed_local_min):
            if i-df.index.start == smoothed_local_min[min_id]:
                smoothed_min_list.append(df['low'][i])
                min_id += 1
            else:
                smoothed_min_list.append(np.nan)

    return smoothed_max_list, smoothed_min_list

In [9]:
def pivot_indexing(smoothed_list):
    x = np.arange(1, len(smoothed_list) + 1, 1)
    y = np.array(smoothed_list)
    x_new=np.flip(x[~np.isnan(y)])
    y_new=np.flip(y[~np.isnan(y)])
    return x_new,y_new   

In [8]:
def detect_trend(x,y):
    ans=101
    for i in range(0,2):
        for j in range(i+2,len(x)):
            LR=LinearRegression()
            LR.fit(x[i:j+1].reshape(-1, 1),y[i:j+1].reshape(-1, 1))
            y_predict= LR.predict(x[i:j+1].reshape(-1, 1))
            temp=(mean_absolute_error(y[i:j+1].reshape(-1, 1),y_predict)/np.mean(y))*100
            #print(temp)
            if(ans>temp or abs(ans-temp)<0.1):
                ans=temp
                values_fit= LR.coef_[0][0],LR.intercept_[0]
                x_best=x[i:j+1]
                y_best=y[i:j+1]
    return values_fit, x_best, y_best
      

In [9]:
def plot_chart(x_max,y_max,x_min,y_min,values_max,values_min,st):
    title = f"Trend Analysis"
    symbol='SPY'
    fig = make_subplots(rows=1, cols=1, subplot_titles=[title])

    fig.add_trace(go.Scatter(x=smoothed_df_close.index-st, y=smoothed_df_close.values, line=dict(color="blue", width=1),name="Smoothed Close"), row=1, col=1)
    fig.add_trace(go.Scatter(x=smoothed_df_low.index-st, y=smoothed_df_low.values, line=dict(color="orange", width=1),name="Smoothed Low"), row=1, col=1)
    fig.add_trace(go.Scatter(x=smoothed_df_high.index-st, y=smoothed_df_high.values, line=dict(color="aquamarine", width=1),name="Smoothed High"), row=1, col=1)
    #Trend Lines
    y_min_extrema= values_min[0]*(smoothed_df_close.index-st) + values_min[1]
    fig.add_trace(go.Scatter(x=smoothed_df_close.index-st, y=y_min_extrema, line=dict(color="red", width=1),name="Min Extrema Trend"), row=1, col=1)

    y_max_extrema = values_max[0]*(smoothed_df_close.index-st) + values_max[1]
    fig.add_trace(go.Scatter(x=smoothed_df_close.index-st, y=y_max_extrema, line=dict(color="green", width=1),name="Max Extrema Trend"), row=1, col=1)

    #x = np.arange(0, len(smoothed_min_list),1)
    fig.add_trace(go.Scatter(x=x_min, y=y_min, marker=dict(color='red', size=7), mode='markers', name='Min extrema'), row=1, col=1)
    fig.add_trace(go.Scatter(x=x_max, y=y_max, marker=dict(color='green', size=7), mode='markers', name='Max extrema'), row=1, col=1)

    fig.update_layout(
          title={'text': f"{symbol} Price", 'x': 0.5},
          autosize=True,
          width=1000, height=700,
          xaxis={"rangeslider": {"visible": False}},
          plot_bgcolor="#EFF5F8")
    fig.update_yaxes(visible=False, secondary_y=True)

    fig.show()

In [15]:
def micro_strategy(symbol,st,end):
    df=load_data(symbol)
    df=df[['date','close','open','high','low']]
    df1= df[st:end]
    global smoothed_df_close,smoothed_df_high,smoothed_df_low
    smoothed_df_close=smoothed_curve(df['close'],bandwidth='cv_ls')
    smoothed_df_low=smoothed_curve(df['low'],bandwidth='cv_ls')
    smoothed_df_high=smoothed_curve(df['high'],bandwidth='cv_ls')
    smoothed_max_list, smoothed_min_list=find_extrema(df1)
    x_new_max,y_new_max=pivot_indexing(smoothed_max_list)
    x_new_min,y_new_min=pivot_indexing(smoothed_min_list)
    values_max,x_max,y_max=detect_trend(x_new_max,y_new_max)
    values_min,x_min,y_min=detect_trend(x_new_min,y_new_min)
    plot_chart(x_max,y_max,x_min,y_min,values_max,values_min,st)
    

    
if __name__ == '__main__':
    symbol = 'NVDA'
    #df = 
    micro_strategy(symbol,658,1258)

In [22]:
#Prepare for Notebook Execution.
#Rolling function Code awaiting.
#Implement RANSAC. (Test on tickers sent)

In [24]:
#RANSAC testing
#combined = np.vstack((x_new_min, y_new_min)).T

In [4]:
def recent_inc_max(X,y):
      #print(X)
    if(X in x_new_max[:2] and len(X)>2):
      #print('Yes')
       return True
    else:
      #print('No')
       return False

def recent_inc_min(X,y):
     #print(X)
    if(X in x_new_min[:2] and len(X)>2):
        #print('Yes')
        return True
    else:
        #print('No')
        return False

In [5]:
def RANSAC(x,y,recent_inc):
    ransac= RANSACRegressor(random_state=42,residual_threshold=1)
    ransac.fit(x.reshape(-1,1),y.reshape(-1,1))
    print(ransac.score(x[ransac.inlier_mask_].reshape(-1,1), y[ransac.inlier_mask_].reshape(-1,1)))
    m, c= ransac.estimator_.coef_,ransac.estimator_.intercept_
    print(ransac.n_skips_invalid_data_)
    return m,c,ransac.inlier_mask_

In [16]:
def plot_strategy(st,x_max,y_max,x_min,y_min,m_max,c_max,m_min,c_min,ransac_max_inlier,ransac_min_inlier):
    title = f"Trend Analysis"
    symbol='SPY'
    fig = make_subplots(rows=1, cols=1, subplot_titles=[title])

    fig.add_trace(go.Scatter(x=smoothed_df_close1.index-st, y=smoothed_df_close1.values, line=dict(color="blue", width=1),name="Smoothed Close"), row=1, col=1)
    fig.add_trace(go.Scatter(x=smoothed_df_low1.index-st, y=smoothed_df_low1.values, line=dict(color="orange", width=1),name="Smoothed Low"), row=1, col=1)
    fig.add_trace(go.Scatter(x=smoothed_df_high1.index-st, y=smoothed_df_high1.values, line=dict(color="aquamarine", width=1),name="Smoothed High"), row=1, col=1)
    #Trend Lines
    y_min_extrema= (m_min[0][0])*(smoothed_df_close1.index-st) + c_min[0]
    fig.add_trace(go.Scatter(x=smoothed_df_close1.index-st, y=y_min_extrema, line=dict(color="red", width=1),name="Min Extrema Trend"), row=1, col=1)

    y_max_extrema = (m_max[0][0])*(smoothed_df_close1.index-st) + c_max[0]
    fig.add_trace(go.Scatter(x=smoothed_df_close1.index-st, y=y_max_extrema, line=dict(color="green", width=1),name="Max Extrema Trend"), row=1, col=1)

    #x = np.arange(0, len(smoothed_min_list),1)
    fig.add_trace(go.Scatter(x=x_min[ransac_min_inlier], y=y_min[ransac_min_inlier], marker=dict(color='red', size=7), mode='markers', name='Min extrema'), row=1, col=1)
    fig.add_trace(go.Scatter(x=x_max[ransac_max_inlier], y=y_max[ransac_max_inlier], marker=dict(color='green', size=7), mode='markers', name='Max extrema'), row=1, col=1)
    #fig.add_trace(go.Scatter(x=x_new_min[~ransac_min.inlier_mask_], y=y_new_min[~ransac_min.inlier_mask_], marker=dict(color='blue', size=7), mode='markers', name='Max extrema'), row=1, col=1)

    fig.update_layout(
          title={'text': f"{symbol} Price", 'x': 0.5},
          autosize=True,
          width=1000, height=600,
          xaxis={"rangeslider": {"visible": False}},
          plot_bgcolor="#EFF5F8")
    fig.update_yaxes(visible=False, secondary_y=True)

    fig.show()

In [18]:
def macro_strategy(symbol,st,end):
    df=load_data(symbol)
    df=df[['date','close','open','high','low']]
    df1= df[st:end]
    global smoothed_df_close1,smoothed_df_high1,smoothed_df_low1
    global x_new_max, x_new_min,y_new_max,y_new_min
    smoothed_df_close1=smoothed_curve(df['close'],bandwidth='cv_ls')
    smoothed_df_low1=smoothed_curve(df['low'],bandwidth='cv_ls')
    smoothed_df_high1=smoothed_curve(df['high'],bandwidth='cv_ls') 
    smoothed_max_list, smoothed_min_list=find_extrema(df1)
    x_new_max, y_new_max=pivot_indexing(smoothed_max_list)
    x_new_min,y_new_min=pivot_indexing(smoothed_min_list)
    
    m_min, c_min,ransac_min_inlier= RANSAC(x_new_min.reshape(-1,1),y_new_min.reshape(-1,1),recent_inc_min)
    
    m_max, c_max,ransac_max_inlier= RANSAC(x_new_max.reshape(-1,1),y_new_max.reshape(-1,1),recent_inc_max)
    
    plot_strategy(st,x_new_max,y_new_max,x_new_min,y_new_min,m_max,c_max,m_min,c_min,ransac_max_inlier,ransac_min_inlier)
    
if __name__ == '__main__':
    procs = []
    symbol = 'NVDA'
    #df = 
    macro_strategy(symbol,658,1258)

0.9995567188887825
0
0.9999190308681297
0


In [ ]:
#Find top k(say 5) lines through ransac